## Working with Memories to Store User Context Across Sessions

### Installing Required Libraries

In [ ]:
%pip install azure-ai-projects==2.0.0b2 openai==1.109.1 python-dotenv azure-identity

### Setting up the Environment Variables

In [ ]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
load_dotenv()

foundry_project_endpoint = os.getenv("FOUNDRY_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")
embedding_model_name = os.getenv("TEXT_EMBEDDING_MODEL_NAME")


print("embedding_model_name:", embedding_model_name)

### Creating the Foundry Project Client

In [ ]:
client = AIProjectClient(
    endpoint=foundry_project_endpoint,
    credential=DefaultAzureCredential()
)

### Defining the Memory Store

In [ ]:
from azure.ai.projects.models import MemoryStoreDefaultDefinition, MemoryStoreDefaultOptions

# Define the Memory Store
definition = MemoryStoreDefaultDefinition(
    chat_model = model_deployment_name,
    embedding_model=embedding_model_name,
    options = MemoryStoreDefaultOptions(
        user_profile_enabled=True,
        chat_summary_enabled=True
    )
)

# Create the Memory Store
memory_store = client.memory_stores.create(
    name = "demo_memory_store_v1",
    definition = definition,
    description = "A memory store to retain user context across sessions"
)

print("Memory Store created with name:", memory_store.name)

### Creating a Function to Update the Memory Store

In [ ]:
from azure.ai.projects.models import ResponsesUserMessageItemParam

def update_memory_store(memory_store_name: str, user_name: str, user_message: str):
    try:
        
        # set the scope to associate the memories with
        scope = user_name

        user_message_item = ResponsesUserMessageItemParam(
            content = user_message
        )

        update_poller = client.memory_stores.begin_update_memories(
            name = memory_store_name,
            scope = scope,
            items = [user_message_item],
            update_delay=0
        )

        # Wait for the update operation to complete
        update_result = update_poller.result()

        print(f"Updated with {len(update_result.memory_operations)} memory operations")

        for operation in update_result.memory_operations:
            print(
                f"  - Operation: {operation.kind}, Memory ID: {operation.memory_item.memory_id}, Content: {operation.memory_item.content}"
        )
        
        return "updated successfully"
    except Exception as e:
        print("Error updating memory store:", str(e))
        return "update failed"

In [ ]:
update_memory_store(
    memory_store_name = memory_store.name,
    user_name = "kuljot",
    user_message = "My name is Kuljot Singh and I love programming in Python. I am also allergic to peanuts and dairy."
)

### Creating the Agent in Foundry Agent Service

In [ ]:
from azure.ai.projects.models import PromptAgentDefinition

agent_name = "helpful-assistant-agent"

agent = client.agents.create_version(
    agent_name=agent_name,
    definition=PromptAgentDefinition(
        model=model_deployment_name,
        instructions="You are a helpful assistant. Respond to all queries in a friendly and informative manner.",
    ),
)

# printing the agent id
print(f"Agent created (id: {agent.id}, name: {agent.name}, version: {agent.version})")

### Creating a Chat System to See the Memories in Action

In [ ]:
from azure.ai.projects.models import MemorySearchOptions

chat = True

user_name = input("Enter the user name (or type 'exit' to quit): ")

while chat:
    user_message = input("Type \"exit\" to quit the chat, \"update\" to update memory store with agent response. or ask your question.")

    if user_message.lower() == 'exit':
        chat = False
    # Only update memory store if user entered "UPDATE"
    if user_message.strip().upper() == "UPDATE":
            user_query = input("Enter something that you want to add to the memory store:")
            update_memory_store(
                memory_store_name = memory_store.name,
                user_name = user_name,
                user_message = user_query
            )
    else:
        # creating the openai client
        openai_client = client.get_openai_client()

        # searching for memories and appending to the user message as grounding context
        query_message = ResponsesUserMessageItemParam(content = user_message)

        search_response = client.memory_stores.search_memories(
            name = memory_store.name,
            scope = user_name,
            items = [query_message],
            options = MemorySearchOptions(max_memories=5)
        )
        
        # appending the found memories to the user message to create grounding context
        if len(search_response.memories) > 0:
            for memory in search_response.memories:
                print(f"Found Memory: {memory.memory_item.content}")
                user_message += f"\nMemory Context: {memory.memory_item.content}"

        response = openai_client.responses.create(
            extra_body = {
                "agent": {
                    "name": agent_name,
                    "type": "agent_reference"
                }
            },
            input = user_message
        )

        agent_response = response.output_text

        print(f"Agent: {agent_response}\n")